# Lecture 2 - 07.06.2021 

# Exercise: Azimuth compression, part 2

In [18]:
# --- To get started, we set the ipympl backend, which makes matplotlib plots interactive.
# --- We do this using a magic command, starting with %.
%matplotlib widget

# --- import libraries

import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq, fftshift, ifft

# --- import rat class
from ste_io import *

# --- create useful functions

def rect(t, T0):
    # Implements y = rect(t/T0) = 1 if abs(t/T0) <= 0.5, 0 otherwise 
    # All input / outputs are Numpy arrays
    
    N = t.shape[0]
    y = np.zeros(N, 'float32') 
    y[np.abs(t/T0) <= 0.5] = 1
    
    return y



Consider the following input parameters (DLR's F-SAR platform, P-band, AfriSAR campaign):


In [17]:

# --- Inputs

# speed of light
c0 = 2.9972190e+08
# SAR wavelength
lambda0 = 0.68901587

# --- parameters for range focusing

# range chirp duration, s
tau = 1.0016000e-05
# chirp bandwidth, Hz
Wr  = 50.0e6
# range sampling, m
rsp = 1.1988876
# number of samples 
Nsr = 3772
# range delay, s <---- NEW !!!
rd  = 4.3008000e-05
# the range delay is the amount of time between the chirp transmission
# and the receipt of the first backscattered signal

# --- parameters for azimuth focusing

# azimut resolution
deltasa = 1.5
# platform velocity, meters/second
vp      = 102.89772    
# range at closest point
R0      = 6445.2197      
# pixel spacing in meters
azsp    = 0.90879266    
# number of pixels 
Nsaz    = 9984


Consider now the raw data in ./data/01-sar/raw-img.sar-processor.range-dep-chirp.hamming.rat :

In [15]:

img = rrat('./data/01-sar/raw-img.sar-processor.range-dep-chirp.hamming.rat')
print(img.shape)

plt.figure()
plt.imshow(abs(img), vmin = 0, vmax = 2*np.mean(np.abs(img)), cmap = 'gray')
plt.title('Raw data')


(9984, 3772)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Raw data')

**Now focus in both range and azimuth !**

In both cases, the matched filters in range and azimuth have to be defined, and applied to the data (*for* loops in both the range and azimuth dimensions).

* Range focusing : 


In [14]:

# generation of the range time axis
Tdr   = (Nsr - 1) * rsp *2 / c0           # total duration
rtime = np.linspace(-Tdr/2, Tdr/2, Nsr)   
# chirp rate in range
kr = Wr / tau

# generate chirp
rchirp = np.exp(1j*np.pi*kr*(rtime)**2) * rect(rtime, tau)

# matched filter
matched_filter_r = np.conj(rchirp[::-1])
matched_filter_r_spectrum = fft(matched_filter_r)

# focused image in range
imgr = np.zeros((img.shape), 'complex64')
# image with the range chirp
imrchirp = np.zeros((img.shape), 'float32')

for nn in range(img.shape[0]):
    
    imrchirp[nn, :] = np.real(rchirp)
    imgr[nn, :] = fftshift(ifft(fft(img[nn, :]) *  matched_filter_r_spectrum))




* Azimuth focusing :

Hint: adapt the procedure used last time to a range-varying azimuth illumination time !